In [ ]:
import os
import subprocess
import sys
import platform
import shutil

def is_ffmpeg_installed():
    return shutil.which("ffmpeg") is not None

def install_ffmpeg():
    system = platform.system()
    print("⚠️ FFmpeg non trovato. Avvio installazione...")
    if system == "Windows":
        import zipfile, urllib.request, tempfile
        temp_dir = tempfile.mkdtemp()
        zip_path = os.path.join(temp_dir, "ffmpeg.zip")
        url = "https://www.gyan.dev/ffmpeg/builds/ffmpeg-release-essentials.zip"
        urllib.request.urlretrieve(url, zip_path)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        bin_dir = next(
            (os.path.join(temp_dir, d, "bin") for d in os.listdir(temp_dir) if d.startswith("ffmpeg")), None
        )
        if bin_dir:
            os.environ["PATH"] += os.pathsep + bin_dir
        else:
            print("❌ Errore durante l’estrazione.")
            sys.exit(1)
    elif system == "Darwin":
        subprocess.run(["brew", "install", "ffmpeg"], check=True)
    elif system == "Linux":
        subprocess.run(["sudo", "apt", "update"], check=True)
        subprocess.run(["sudo", "apt", "install", "-y", "ffmpeg"], check=True)
    else:
        print("❌ Sistema non supportato.")
        sys.exit(1)

def convert_to_webm_2pass(input_path, output_path, target_bitrate='4M'):
    log_file = 'ffmpeg2pass.log'
    cmd_pass1 = [
        'ffmpeg', '-y', '-i', input_path,
        '-c:v', 'libvpx-vp9', '-b:v', target_bitrate,
        '-pass', '1', '-an', '-f', 'null', os.devnull
    ]
    cmd_pass2 = [
        'ffmpeg', '-y', '-i', input_path,
        '-c:v', 'libvpx-vp9', '-b:v', target_bitrate,
        '-pass', '2', '-c:a', 'libopus', output_path
    ]
    print(f"🎬 Primo passaggio: {input_path}")
    subprocess.run(cmd_pass1, check=True)
    print(f"🎬 Secondo passaggio: {input_path} → {output_path}")
    subprocess.run(cmd_pass2, check=True)
    for f in [log_file, log_file + '.log.mbtree']:
        if os.path.exists(f):
            os.remove(f)

def batch_convert(folder_path, target_bitrate='4M'):
    for file in os.listdir(folder_path):
        if file.lower().endswith('.mp4'):
            input_path = os.path.join(folder_path, file)
            output_path = os.path.join(folder_path, os.path.splitext(file)[0] + '.webm')
            convert_to_webm_2pass(input_path, output_path, target_bitrate)

if not is_ffmpeg_installed():
    install_ffmpeg()

folder = input("video").strip()
if os.path.isdir(folder):
    batch_convert(folder, target_bitrate='4M')
    print("✅ Conversione completata.")
else:
    print("❌ Percorso non valido.")

❌ Percorso non valido.
